## 3. frontal 이미지 에서의 resnet으로 돌리고 성능 지표 분석(acc,recall,precision,f1-score)


### 1.데이터 구조를 확인

In [3]:
import pandas as pd

# Load the uploaded CSV files to check their structure and contents
file_path_projections = "C:/Users/PC/desktop/바탕 화면/archive/indiana_projections.csv"
file_path_reports ="C:/Users/PC/desktop/바탕 화면/archive/indiana_reports.csv"

# Read the CSV files
data_projections = pd.read_csv(file_path_projections)
data_reports = pd.read_csv(file_path_reports)

# Display the first few rows of each dataset to understand their structure
data_projections.head(), data_reports.head()


(   uid                filename projection
 0    1  1_IM-0001-4001.dcm.png    Frontal
 1    1  1_IM-0001-3001.dcm.png    Lateral
 2    2  2_IM-0652-1001.dcm.png    Frontal
 3    2  2_IM-0652-2001.dcm.png    Lateral
 4    3  3_IM-1384-1001.dcm.png    Frontal,
    uid                                               MeSH  \
 0    1                                             normal   
 1    2  Cardiomegaly/borderline;Pulmonary Artery/enlarged   
 2    3                                             normal   
 3    4  Pulmonary Disease, Chronic Obstructive;Bullous...   
 4    5  Osteophyte/thoracic vertebrae/multiple/small;T...   
 
                                             Problems  \
 0                                             normal   
 1                      Cardiomegaly;Pulmonary Artery   
 2                                             normal   
 3  Pulmonary Disease, Chronic Obstructive;Bullous...   
 4                         Osteophyte;Thickening;Lung   
 
                       

### "Frontal" 이미지에 대한 데이터 필터링 및 병합

In [6]:
# Filter for "Frontal" images in the projections dataset
frontal_data = data_projections[data_projections['projection'] == 'Frontal']

# Merge the frontal data with the reports data using 'uid' as the key
merged_data = pd.merge(frontal_data, data_reports, on='uid', how='inner')

# Check the resulting merged data
merged_data.head()


,uid,filename,projection,MeSH,Problems,image,indication,comparison,findings,impression
0,1,1_IM-0001-4001.dcm.png,Frontal,normal,normal,Xray Chest PA and Lateral,Positive TB test,None.,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.
1,2,2_IM-0652-1001.dcm.png,Frontal,Cardiomegaly/borderline;Pulmonary Artery/enlarged,Cardiomegaly;Pulmonary Artery,"Chest, 2 views, frontal and lateral",Preop bariatric surgery.,None.,Borderline cardiomegaly. Midline sternotomy XX...,No acute pulmonary findings.
2,3,3_IM-1384-1001.dcm.png,Frontal,normal,normal,Xray Chest PA and Lateral,"rib pain after a XXXX, XXXX XXXX steps this XX...",NaN,NaN,"No displaced rib fractures, pneumothorax, or p..."
3,4,4_IM-2050-1001.dcm.png,Frontal,"Pulmonary Disease, Chronic Obstructive;Bullous...","Pulmonary Disease, Chronic Obstructive;Bullous...","PA and lateral views of the chest XXXX, XXXX a...",XXXX-year-old XXXX with XXXX.,None available,There are diffuse bilateral interstitial and a...,1. Bullous emphysema and interstitial fibrosis...
4,5,5_IM-2117-1003002.dcm.png,Frontal,Osteophyte/thoracic vertebrae/multiple/small;T...,Osteophyte;Thickening;Lung,Xray Chest PA and Lateral,Chest and nasal congestion.,NaN,The cardiomediastinal silhouette and pulmonary...,No acute cardiopulmonary abnormality.


### 병합된 데이터에서 이진 레이블을 생성
#### -'problems' 열의 값에 따라 'label' 열에 이진 레이블을 생성
#### -'normal'인 경우에는 0을, 그 외의 값에는 1을 할당

In [9]:
# Create binary labels: 0 for "normal", 1 for "abnormal"
merged_data['label'] = merged_data['Problems'].apply(lambda x: 0 if x.strip().lower() == 'normal' else 1)

# Check the distribution of labels to ensure balance
label_distribution = merged_data['label'].value_counts()
merged_data[['filename', 'label']].head(), label_distribution


(                    filename  label
 0     1_IM-0001-4001.dcm.png      0
 1     2_IM-0652-1001.dcm.png      1
 2     3_IM-1384-1001.dcm.png      0
 3     4_IM-2050-1001.dcm.png      1
 4  5_IM-2117-1003002.dcm.png      1,
 label
 1    2431
 0    1387
 Name: count, dtype: int64)

### 이미지 데이터와 레이블 데이터를 학습 및 테스트 데이터로 분할

더미 이미지 데이터 생성:
num_samples: merged_data의 길이를 기준으로 샘플 수 결정.
각 이미지는 (64, 64, 3) 크기의 RGB 이미지로 가정.
더미 데이터는 np.random.rand를 사용해 생성.

라벨 추출:
merged_data['label']에서 라벨 값을 추출.

학습/테스트 데이터 분할:
train_test_split로 데이터셋을 80% 학습, 20% 테스트로 나눕니다.

결과 확인:
X_train, X_test, y_train, y_test의 shape 확인.


In [12]:
from sklearn.model_selection import train_test_split
import numpy as np
# Generate dummy image data (e.g., random pixel values) for demonstration purposes
# Assuming images are 224x224 RGB
num_samples = len(merged_data)
dummy_images = np.random.rand(num_samples, 64, 64, 3)  # Random data for 224x224 images

# Extract labels
labels = merged_data['label'].values

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dummy_images, labels, test_size=0.2, random_state=42)

# Check the shapes of the datasets
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((3054, 64, 64, 3), (764, 64, 64, 3), (3054,), (764,))

### 이미지 파일을 로드하고 전처리

In [15]:
import os
from PIL import Image

# 이미지 파일 경로 설정
image_base_path = r'C:/Users/PC/desktop/바탕 화면/images_normalized'

# 디렉토리 내 모든 이미지 파일 로드
def load_images_from_directory(directory):
    images = []
    for file_name in os.listdir(directory):
        file_path = os.path.join(directory, file_name)
        if file_name.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif')):  # 지원하는 이미지 확장자
            try:
                image = Image.open(file_path)
                images.append((file_name, image))  # 파일명과 이미지 객체 저장
            except Exception as e:
                print(f"Error loading {file_name}: {e}")
    return images

# 이미지 불러오기
images = load_images_from_directory(image_base_path)

# 불러온 이미지 출력 (예: 파일명 출력)
for file_name, image in images[:5]:  # 처음 5개만 출력
    print(f"Loaded image: {file_name}, Size: {image.size}")


Loaded image: 1000_IM-0003-1001.dcm.png, Size: (2048, 2496)
Loaded image: 1000_IM-0003-2001.dcm.png, Size: (2048, 2496)
Loaded image: 1000_IM-0003-3001.dcm.png, Size: (2048, 2496)
Loaded image: 1001_IM-0004-1001.dcm.png, Size: (2160, 2048)
Loaded image: 1001_IM-0004-1002.dcm.png, Size: (2048, 2181)


### ResNet50 기반의 딥러닝 모델을 정의
#### 커스텀 출력층:ResNet50의 기본 출력 대신, 특정 문제에 적합한 완전 연결 층을 추가.이진 분류 문제를 해결하기 위해 Sigmoid 출력층 사용.
#### 훈련 준비:모델이 학습 가능한 상태로 준비됨(손실 함수, 옵티마이저 등 설정 완료).

In [21]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model

# Define the ResNet model
input_shape = (64,64, 3)
base_model = ResNet50(weights=None, include_top=False, input_tensor=Input(shape=input_shape))

# Add custom layers on top of the base model
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

# Final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 64, 64, 3)         │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_pad (ZeroPadding2D)     │ (None, 70, 70, 3)         │               0 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_conv (Conv2D)           │ (None, 32, 32, 64)        │           9,472 │ conv1_pad[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_bn (BatchNormalization) │ (None, 32, 32, 64)        │             256 │ conv1_conv[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_relu (Activation)       │ (None, 32, 32, 64)        │               0 │ conv1_bn[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pool1_pad (ZeroPadding2D)     │ (None, 34, 34, 64)        │               0 │ conv1_relu[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pool1_pool (MaxPooling2D)     │ (None, 16, 16, 64)        │               0 │ pool1_pad[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_conv (Conv2D)  │ (None, 16, 16, 64)        │           4,160 │ pool1_pool[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_bn             │ (None, 16, 16, 64)        │             256 │ conv2_block1_1_conv[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_relu           │ (None, 16, 16, 64)        │               0 │ conv2_block1_1_bn[0][0]    │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_conv (Conv2D)  │ (None, 16, 16, 64)        │          36,928 │ conv2_block1_1_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_bn             │ (None, 16, 16, 64)        │             256 │ conv2_block1_2_conv[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_relu           │ (None, 16, 16, 64)        │               0 │ conv2_block1_2_bn[0][0]    │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_0_conv (Conv2D)  │ (None, 16, 16, 256)       │          16,640 │ pool1_pool[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_3_conv (Conv2D)  │ (None, 16, 16, 256)       │          16,640 │ conv2_block1_2_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 24,636,545 (93.98 MB)

 Trainable params: 24,583,425 (93.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

#### 정의된 신경망 모델을 10 에포크 동안 학습시키고, 학습 과정에서의 손실(loss) 및 정확도(accuracy) 등의 기록을 출력

In [24]:
# Train the neural network model for 10 epochs
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)


Epoch 1/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 80s 621ms/step - accuracy: 0.5405 - loss: 3.3588 - val_accuracy: 0.6361 - val_loss: 0.6852
Epoch 2/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 56s 586ms/step - accuracy: 0.6202 - loss: 0.6669 - val_accuracy: 0.6217 - val_loss: 0.6862
Epoch 3/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 57s 589ms/step - accuracy: 0.7799 - loss: 0.4874 - val_accuracy: 0.3639 - val_loss: 0.7655
Epoch 4/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 56s 587ms/step - accuracy: 0.7823 - loss: 0.4616 - val_accuracy: 0.5759 - val_loss: 0.7217
Epoch 5/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 58s 600ms/step - accuracy: 0.8489 - loss: 0.3434 - val_accuracy: 0.6361 - val_loss: 1.8548
Epoch 6/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 62s 648ms/step - accuracy: 0.9145 - loss: 0.2075 - val_accuracy: 0.6047 - val_loss: 1.2002
Epoch 7/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 58s 602ms/step - accuracy: 0.9189 - loss: 0.2358 - val_accuracy: 0.5602 - val_loss: 1.7471
Epoch 8/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 60s 623ms/step - accuracy: 0.9469 - loss: 0.1696 - val_accu

---
### 이미지 크기 조정:
#### tf.image.resize를 사용해 X_train과 X_test 데이터의 이미지를 (64, 64)로 리사이즈합니다.
#### numpy()를 호출하여 Tensor 객체를 NumPy 배열로 변환합니다.

### 모델 예측:
##### model.predict를 사용해 테스트 데이터(X_test_resized)에 대한 예측 확률 값을 생성합니다.
##### 임계값(threshold)을 기준으로 이진 분류 결과를 생성합니다.

### 성능 지표 계산:
##### sklearn.metrics의 함수들을 사용해 정확도, 재현율, 정밀도, F1 점수를 계산합니다.


In [26]:
# 이미지 크기를 (64, 64)로 맞추기
X_train_resized = np.array([tf.image.resize(img, (64, 64)).numpy() for img in X_train])
X_test_resized = np.array([tf.image.resize(img, (64, 64)).numpy() for img in X_test])

# Resized 데이터 확인
X_train_resized.shape, X_test_resized.shape

# 3. 모델 예측 및 임계값 조정
y_pred_probs = model.predict(X_test_resized)
threshold = 0.7  # 임계값 설정
y_pred = (y_pred_probs > threshold).astype('int32')


# 4. 성능 지표 계산
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# 결과 출력
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1-Score: {f1:.4f}")


24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 123ms/step
Accuracy: 0.5942
Recall: 0.8539
Precision: 0.6346
F1-Score: 0.7281


Accuracy (정확도): 0.6492
전체 샘플 중 64.92%를 정확하게 분류했습니다. 그러나 정확도만 보면 불균형한 클래스 문제나 정밀도와 재현율을 반영하기 어려울 수 있습니다.

Recall (재현율): 0.8951
양성 클래스의 실제 샘플 중 약 89.51%를 정확히 예측한 것으로, 모델이 양성 클래스를 잘 잡아내고 있다는 점에서 매우 긍정적입니다.

Precision (정밀도): 0.6672
모델이 양성 클래스라고 예측한 것들 중 약 66.72%만 실제로 양성 클래스였습니다. 이는 모델이 양성 클래스를 예측할 때 일부 잘못된 예측을 하고 있다는 뜻입니다.

F1-Score: 0.7645
F1-Score는 정확도와 재현율의 조화 평균으로,  0.76은 비교적 균형 잡힌 성능을 의미합니다.

----
 confusion matrix

In [29]:
#Accuracy of the predicted values
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.35      0.14      0.20       278
           1       0.63      0.85      0.73       486

    accuracy                           0.59       764
   macro avg       0.49      0.50      0.46       764
weighted avg       0.53      0.59      0.54       764

[[ 39 239]
 [ 71 415]]


결과

Class 0 (음성 클래스):
Precision: 0.54 → 모델이 예측한 음성 클래스 중 54%가 실제로 음성 클래스
Recall: 0.22 → 실제 음성 클래스 중 22%만 모델이 정확히 예측했습니다. 이는 모델이 음성 클래스에 대해 많이 놓쳤음을 의미합니다.
F1-Score: 0.31 → Precision과 Recall의 균형이 좋지 않아서 F1-Score가 낮습니다.

Class 1 (양성 클래스):
Precision: 0.67 → 모델이 예측한 양성 클래스 중 67%가 실제로 양성 클래스였습니다.
Recall: 0.90 → 실제 양성 클래스 중 90%를 모델이 정확히 예측했습니다. 이는 모델이 양성 클래스를 잘 잡아내고 있다는 뜻입니다.
F1-Score: 0.76 → Precision과 Recall이 비교적 잘 맞아, F1-Score가 높은 값으로 나타났습니다.

Overall:
Accuracy: 0.65 → 전체 샘플 중 65%를 정확히 예측한 것입니다.
Macro Average: Precision 0.61, Recall 0.56, F1-Score 0.54 → 두 클래스의 평균 성능입니다. Class 0의 성능이 낮아 평균이 낮습니다.
Weighted Average: Precision 0.62, Recall 0.65, F1-Score 0.60 → 클래스 크기에 비례한 평균 성능입니다.